In [28]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from sklearn.model_selection import train_test_split


# # Define the attributes
# attributes = [
#     "STATE/UT", "YEAR", "MURDER/ATTEMPT TO MURDER", "RAPE", 
#     "KIDNAPPING & ABDUCTION", "DACOITY", "PREPARATION AND ASSEMBLY FOR DACOITY", 
#     "ROBBERY", "BURGLARY", "THEFT", "RIOTS", "CHEATING", 
#     "COUNTERFEITING", "ARSON", "HURT/GRIEVOUS HURT", "DOWRY DEATHS", 
#     "CRUELTY BY HUSBAND OR HIS RELATIVES"
# ]

# # Define the states/UTs and years
# states_ut = ["Andhra Pradesh", "Arunachal Pradesh", "Assam", "Bihar", "Chhattisgarh", 
#              "Goa", "Gujarat", "Haryana", "Himachal Pradesh", "Jharkhand", 
#              "Karnataka", "Kerala", "Madhya Pradesh", "Maharashtra", "Manipur", 
#              "Meghalaya", "Mizoram", "Nagaland", "Odisha", "Punjab", 
#              "Rajasthan", "Sikkim", "Tamil Nadu", "Telangana", "Tripura", 
#              "Uttar Pradesh", "Uttarakhand", "West Bengal"]
# years = list(range(2005, 2017))  # 12 years of data

# # Generate synthetic data
# data = []
# for state in states_ut:
#     for year in years:
#         row = [
#             state, year,
#             np.random.randint(0, 1000), np.random.randint(0, 1000),
#             np.random.randint(0, 1000), np.random.randint(0, 1000),
#             np.random.randint(0, 1000), np.random.randint(0, 1000),
#             np.random.randint(0, 1000), np.random.randint(0, 1000),
#             np.random.randint(0, 1000), np.random.randint(0, 1000),
#             np.random.randint(0, 1000), np.random.randint(0, 1000),
#             np.random.randint(0, 1000), np.random.randint(0, 1000),
#             np.random.randint(0, 1000)
#         ]
#         data.append(row)

# Load the dataset
df = pd.read_csv('01_District_wise_crimes_committed_IPC_2001_2012.csv')

# Calculate total crimes and categorize them
df['TOTAL_CRIMES'] = df.drop(columns=['YEAR', 'STATE/UT']).sum(axis=1)

# Define bins for categorizing total crimes
bins = [0, df['TOTAL_CRIMES'].quantile(0.33), df['TOTAL_CRIMES'].quantile(0.66), df['TOTAL_CRIMES'].max()]
labels = ['low', 'medium', 'high']
df['CRIME_CATEGORY'] = pd.cut(df['TOTAL_CRIMES'], bins=bins, labels=labels, include_lowest=True)


In [30]:
# Separate features and target variable
X = df.drop(columns=['YEAR', 'STATE/UT', 'TOTAL_CRIMES', 'CRIME_CATEGORY'])
y = df['CRIME_CATEGORY']

# Encode target labels
y = pd.factorize(y)[0]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [32]:
def gini_index(groups, classes):
    # Count all samples at the split point
    n_instances = float(sum([len(group) for group in groups]))
    
    # Sum weighted Gini index for each group
    gini = 0.0
    for group in groups:
        size = float(len(group))
        # Avoid division by zero
        if size == 0:
            continue
        score = 0.0
        # Score the group based on the score for each class
        for class_val in classes:
            proportion = [row[-1] for row in group].count(class_val) / size
            score += proportion * proportion
        # Weight the group score by its relative size
        gini += (1.0 - score) * (size / n_instances)
    return gini
def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right
def get_split(dataset):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for index in range(len(dataset[0]) - 1):
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index': b_index, 'value': b_value, 'groups': b_groups}


In [34]:
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)
def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    # Check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # Check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # Process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    # Process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)
def build_tree(train, max_depth, min_size):
    root = get_split(train)
    split(root, max_depth, min_size, 1)
    return root
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']
# Prepare the data for the model (convert to list of lists)
train_data = np.hstack((X_train, y_train.reshape(-1,1))).tolist()
test_data = np.hstack((X_test, y_test.reshape(-1,1))).tolist()

# Build the tree
max_depth = 5
min_size = 10
tree = build_tree(train_data, max_depth, min_size)

# Make predictions
predictions = [predict(tree, row) for row in X_test]

# Evaluate predictions
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions, average='weighted')
recall = recall_score(y_test, predictions, average='weighted')
f1 = f1_score(y_test, predictions, average='weighted')

print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')


Accuracy: 0.89
Precision: 0.90
Recall: 0.89
F1 Score: 0.89
